# Exercise_1

Basic idea: Choose scenario with click on a button in the GUI. Then click on a Start button to run the simulation. \
    In each timestep of the simulation the new (precise) position of each pedestrian is calculated. Afterwards the discretized position in the grid/cellular automaton is shown in the GUI.

In [1]:
import tkinter as tk
import numpy as np
import math
import time

In [2]:
class Person():
    '''
    pos_x, pos_y: precise position of pedestrian (Idea: cells are discretized positions we only use for drawing)
    speed: in cells/timestep. if timestep=1s, cellsize=0.25m and desiredSpeed=1.6m/s -> speed needs to be 1.6/0.25=6.4 cells/timestep
    move(): see below
    '''
    def __init__(self, position_x, position_y, speed=1):
        self.pos_x = position_x
        self.pos_y = position_y
        self.speed = speed 
        self.leap = 0 # >0 bonus for next step, <0 penalty for next step
        
    def move(self, grid, scenario):
        '''
        equals an update for the position depending on speed, interaction of pedestrians and obstacle avoidance
        costField is discritized per se, so position needs to be discretized as well
        looks for minimal cost in all directions within the feasible area
        '''
        self.pedestrianInteraction(scenario)#updates interactionField
        costField = np.array(scenario.costField) + np.array(self.interactionField)# + np.array(scenario.obstacleField)
        ## for debugging of costField, interactionField .. 
        #for (x,y),v in np.ndenumerate(np.array(self.interactionField)):
        #    if v > 0:
        #        print('('+str(x),str(y)+')'+': '+str(v))
        #        print('cost single'+': '+str(scenario.costField[x][y]))
        #        print('cost combined'+': '+str(costField[x][y]))
        ##
        (x_opt, y_opt) = (self.pos_x, self.pos_y)
        cost_min = scenario.costField[round(self.pos_x)][round(self.pos_y)]
        
        #assuming constant timesteps of 1s
        pos_dist = self.leap + self.speed #possible distance for the pedestrian to move in this timestep
        if round(pos_dist) <= 0:#pedestrian needs to wait in order to move with correct speed
            self.leap += self.speed
            return
            
        while round(pos_dist) > 0:#if move at this timestep 
            #steps only +-1
            dxplus = self.pos_x + 1
            dxminus = self.pos_x - 1
            dyplus = self.pos_y + 1
            dyminus = self.pos_y - 1

            dxx = (self.pos_x, dxplus, dxminus)
            dyy = (self.pos_y, dyplus, dyminus)

            cell_move = (cost_min, self.pos_x, self.pos_y)             

            flag_diag = False #indicates wether the pedestrian moves diagonal or not
            
            for x in dxx:
                for y in dyy:
                    if scenario.isInDomain(x,y):
                        if (x,y) == (self.pos_x, self.pos_y):
                            continue#no need to check own position again
                        else:
                            if grid.grid[x][y] == 'E':#neighbored cell is empty
                                cell_dist = costField[x][y]
                                if  cell_dist < cell_move[0]:
                                    cell_move = (cell_dist,x,y)
                                    if ((self.pos_x - x)+(self.pos_y - y))%2 == 0:#diagonal movement yes/no
                                        flag_diag = True
                                    else:
                                        flag_diag = False
            
            #updates grid for drawing and own position
            grid.grid[round(self.pos_x)][round(self.pos_y)]='E'
            grid.grid[round(cell_move[1])][round(cell_move[2])]='P'
            self.pos_x = cell_move[1]
            self.pos_y = cell_move[2]
            
            #calculate new (remaining) possible distance depending on  (striaght/diag) and update self leap
            if flag_diag == True:
                pos_dist -= math.sqrt(2)
                self.leap = pos_dist
            else:
                pos_dist -= 1
                self.leap = pos_dist        
        
        
    def pedestrianInteraction(self, scenario):#TODO: needs isInDomain() check
        self.interactionField = [[0.0 for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            if p == self:
                continue#this is to avoid penalty with yourself. I assume you like yourself :)
            else:
                x = round(p.pos_x)
                y = round(p.pos_y)
                dxx = [x]
                dyy = [y]
                for r in range(scenario.rmax)[1:]:#rmax = 3 -> (0,)1,2
                    dxx.append(x+r)
                    dxx.append(x-r)
                    dyy.append(y+r)
                    dyy.append(y-r)
                for xx in dxx:
                    for yy in dyy:
                        if scenario.isInDomain(xx,yy):
                            if (xx - x)**2 + (yy - y)**2 >= scenario.rmax**2:#cell not within cirlce around pedestrian
                                continue
                            r = math.sqrt((x - xx)**2 + (y - yy)**2)#eucliddistance between p.pos_xy and xxyy
                            self.interactionField[xx][yy] += math.exp(1/(r**2 - scenario.rmax**2))

In [7]:
class Scenario():
    '''
    creates persons, targets, obstacles and grid size depending on the scenario
    rmax: maximum radius of pedestrian interaction. Depends on cellsize -> adjust for each scenario
    '''
    def __init__(self):
        self.rows = 25
        self.columns = 25
        self.persons = []
        self.targets = []
        self.obstacles = []
        self.grid = Grid(self)
        self.rmax = 3
        self.fastMarching = False # bottleneck and chickentest both once without Dijkstra/fastMarching and once with
        #self.setTo1()
        
    def setTo1(self):#one person straight
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo2(self):#five persons in a circle
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 35), Person(25, 15), Person(40, 30), Person(40, 20)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo3(self):#bottleneck
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 5), Person(10, 10), Person(21, 20), Person(39, 30), Person(48, 48)]
        self.targets = [(2, 2)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo3wD(self):#bottleneck with Dijkstra
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 5), Person(10, 10), Person(21, 20), Person(39, 30), Person(48, 48)]
        self.targets = [(2, 2)]
        self.obstacles = []
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo4(self):#chickentest
        self.rows = 25
        self.columns = 25
        self.persons = [Person(6, 12)]
        self.targets = [(18, 12)]
        self.obstacles = [(12,12),(12,13),(12,11),(12,14),(12,10),(12,15),(12,9),(11,15),(10,15),(11,9),(10,9)]
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo4wD(self):#chickentest with Dijkstra
        self.rows = 25
        self.columns = 25
        self.persons = [Person(6, 12)]
        self.targets = [(18, 12)]
        self.obstacles = [(12,12),(12,13),(12,11),(12,14),(12,10),(12,15),(12,9),(11,15),(10,15),(11,9),(10,9)]
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo5(self):#added 5th scenario to test hypothesis
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 45), Person(10, 47), Person(3, 42), Person(18, 43), Person(46, 44), Person(38,41), Person(45,44), Person(45,42)]
        self.targets = [(25, 0)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def rimea1(self):#RiMEA Scenario 1: Straight Line Corridor
        self.rows = 50
        self.columns = 50
        self.persons = [Person(8, 25, 1.33)]
        self.targets = [(48, 25)]
        list = [(49,25),(49,24)]
        for i in range(8,50):
            list.append((i,26))
            list.append((i,23))
        self.obstacles = list
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
    
    #TODO: add RiMEA tests
    #TODO: change persons, obstacles and maybe targets for scenario bottleneck
    #TODO: adjust rmax for each scenario, especially for the RiMEA scenarios
        
    def setupCostField(self):
        #cost scaled by max possible cost/distance
        if self.fastMarching == False:#rudimentary obstacle avoidance only
            self.costField = [[1.0 for y in range(self.columns)] for x in range(self.rows)]
            max_dist = math.sqrt(self.rows**2 + self.columns**2)
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.targets:
                        self.costField[x][y] = 0
                    elif (x,y) in self.obstacles:
                        self.costField[x][y] = math.exp(10)
                    else:#computes cost/distance for nearest target
                        dist = max_dist
                        for t in self.targets:
                            min_dist = math.sqrt((x - t[0])**2 + (y - t[1])**2)
                            if min_dist < dist:
                                dist = min_dist
                        cost = dist#/max_dist # scaling by max_dist can cause pedestrians to stop before the target
                        self.costField[x][y] = cost
        else:#dijkstra or fastmarching
            self.Dijkstra()
            #self.fastMarching()
            
    def Dijkstra(self):
        #initialisation
        self.costField = [[np.inf for y in range(self.columns)] for x in range(self.rows)]
        Q = []
        for x in range(self.rows):
            for y in range(self.columns):
                if (x,y) in self.targets:
                    self.costField[x][y] = 0
                Q.append((x,y))
        #loop
        while len(Q) > 0:
            #find minimal value in Q
            x_min = Q[0][0]
            y_min = Q[0][1]
            for (x,y) in Q:
                if self.costField[x][y] < self.costField[x_min][y_min]:
                    (x_min, y_min) = (x,y)
            Q.remove((x_min,y_min))
            print('Remaining values to calculate: '+str(len(Q)))
            #neighbors of x_min,y_min
            neighbors = [(x_min+1,y_min),(x_min,y_min+1),(x_min-1,y_min),(x_min,y_min-1),(x_min+1,y_min+1),(x_min+1,y_min-1),(x_min-1,y_min-1),(x_min-1,y_min+1)]
            for (nx,ny) in neighbors:
                if self.isInDomain(nx,ny):
                    #update distance for neighbors
                    if (nx,ny) in Q:
                        if (nx,ny) in self.obstacles:
                            self.costField[nx][ny] = math.exp(10)
                        else:
                            alternativ = self.costField[x_min][y_min] + math.sqrt((nx - x_min)**2 + (ny - y_min)**2)
                            if alternativ < self.costField[nx][ny]:
                                self.costField[nx][ny] = alternativ
                            
    def isInDomain(self,x,y):
        if 0 <= x < self.rows and 0 <= y < self.columns:
            return True
        else:
            return False
        
    def fastMarching(self):
        self.obstacleField = [[0.0 for y in range(self.columns)] for x in range(self.rows)]
        if self.fastMarching == False:#rudimentary obstacle avoidance only
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.obstacles:
                        self.obstacleField[x][y] = math.exp(10)
        else:#Fast Marching algorithm
            # waiting for the feedback of Dr. Dietrich since I believe this should not be in the obstacle section
            # That's what Ludwig is working on
            return
            accepted = []
            far = []
            considered = []
            #step 1
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.obstacles:
                        self.obstacleField[x][y] = 0
                        accepted.append((x,y))
                    else:
                        self.obstacleField[x][y] = np.inf
                        far.append((x,y))
            #step 2
            for (x,y) in far:
                U = 1#some new value with update formula
                if U < self.obstacleField[x][y]:
                    self.obstacleField[x][y] = U
                    far.remove((x,y))
                    considered.append((x,y))   
            #steps 3-5
            while len(considered) > 0:
                #step 3
                v = np.inf
                for (x,y) in considered:
                    if self.obstacleField[x][y] < v:
                        v = self.obstacleField[x][y]
                        x_tilda = x
                        y_tilda = y
                considered.remove((x_tilda,y_tilda))
                accepted.append((x_tilda,y_tilda))
                #step 4
                neighbors = [(x_tilda+1,y_tilda),(x_tilda-1,y_tilda),(x_tilda,y_tilda+1),(x_tilda,y_tilda-1)]#diagonal neighbors too?
                for (x,y) in neighbors:
                    U = 1#update formula
                    if U < self.obstacleField[x][y]:
                        self.obstacleField[x][y] = U
                    #step 5
                    if (x,y) in far:
                        far.remove((x,y))
                        considered.append((x,y))
                
    def move(self):
        for p in self.persons:
            p.move(self.grid, self)

In [8]:
class Grid():
    '''
    Basically only for visualization.
    '''
    def __init__(self, scenario):
        self.scenario = scenario
        self.grid = [['E' for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            self.grid[p.pos_x][p.pos_y] = 'P'
        for t in scenario.targets:
            self.grid[t[0]][t[1]] = 'T'
        for o in scenario.obstacles:
            self.grid[o[0]][o[1]] = 'O'
        
    def draw(self, canvas, line_distance):
        '''
        Sould draw the grid and the persons/targets/obstacles.
        Depends on tkinter stuff. I have to look at this in more detail.
        '''
        # vertical lines
        for x in range(line_distance,canvas.winfo_width(),line_distance):
            canvas.create_line(x, 0, x, canvas.winfo_height(), fill="#476042")
        # horizontal lines
        for y in range(line_distance,canvas.winfo_height(),line_distance):
            canvas.create_line(0, y, canvas.winfo_width(), y, fill="#476042")
        # P, T, O or nothing in between the lines
        for (x, y), value in np.ndenumerate(self.grid):
            if value=='P':
                color = 'green'
            elif value=='T':
                color = 'red'
            elif value=='O':
                color = 'black'
            else:
                value=''
                color = 'white'
            canvas.create_text((x+0.5) * line_distance, (y+0.5) * line_distance, text=value, fill=color)
            # canvas.create_rectangle(..) as possible improvement for visualization

In [9]:
def main():
    '''
    Setup of the interface stuff.
    When a scenario is selected via a button, it should appear next to the buttons in the canvas.
    When clicking on start button, the current scenario should be simulated, i.e. for every timestep=1s each pedestrian should move() and the drawing should be renewed.
    
    '''
    window = tk.Tk()
    window.title('Exercise 1')
    
    scenario = Scenario()
    
    #(TODO:) make line_distance dependend on scenario columns/rows for nicer layout. Same for fontsize?
    line_distance = 20#should be passed to draw() in Grid
    canvas_width = line_distance * scenario.columns
    canvas_height = line_distance * scenario.rows
    canvas = tk.Canvas(master=window, width=canvas_width, height=canvas_height)
    canvas.pack(side='left')
    label = tk.Label(master=window, text='0 s', width='18')
    label.pack()
    
    btnScenario1 = tk.Button(master=window, text='1 Pedestrian', bg='#26f9ad', width='18', command=lambda: scenario.setTo1())
    btnScenario1.pack()#(side=RIGHT)
    btnScenario2 = tk.Button(master=window, text='Pedestrians in a circle', bg='#26f9ad', width='18', command=lambda: scenario.setTo2())
    btnScenario2.pack()#(side=RIGHT)
    btnScenario3 = tk.Button(master=window, text='Bottleneck', bg='#26f9ad', width='18', command=scenario.setTo3)
    btnScenario3.pack()#(side=RIGHT)
    btnScenario3wD = tk.Button(master=window, text='Bottleneck w/ Dijkstra', bg='#26f9ad', width='18', command=scenario.setTo3wD)
    btnScenario3wD.pack()#(side=RIGHT)
    btnScenario4 = tk.Button(master=window, text='Chickentest', bg='#26f9ad', width='18', command=scenario.setTo4)
    btnScenario4.pack()#(side=RIGHT)
    btnScenario4wD = tk.Button(master=window, text='Chickentest w/ Dijkstra', bg='#26f9ad', width='18', command=scenario.setTo4wD)
    btnScenario4wD.pack()#(side=RIGHT)
    btnScenario5 = tk.Button(master=window, text='Scenario 5', bg='#26f9ad', width='18', command=scenario.setTo5)
    btnScenario5.pack()#(side=RIGHT)
    btnScenarioR1 = tk.Button(master=window, text='RiMEA Scenario 1', bg='#26f9ad', width='18', command=scenario.rimea1)
    btnScenarioR1.pack()#(side=RIGHT)
    btnScenarioR4 = tk.Button(master=window, text='RiMEA Scenario 4', bg='#26f9ad', width='18', command=scenario.setTo5)
    btnScenarioR4.pack()#(side=RIGHT)
    btnScenarioR6 = tk.Button(master=window, text='RiMEA Scenario 6', bg='#26f9ad', width='18', command=scenario.setTo5)
    btnScenarioR6.pack()#(side=RIGHT)
    btnScenarioR7 = tk.Button(master=window, text='RiMEA Scenario 7', bg='#26f9ad', width='18', command=scenario.setTo5)
    btnScenarioR7.pack()#(side=RIGHT)
    
    
    def start(window, canvas, scenario, time_limit=50):
        canvas_width = line_distance * scenario.columns
        canvas_height = line_distance * scenario.rows
        canvas.configure(width=canvas_width, height=canvas_height)
        canvas.delete('all')
        window.after(10, lambda: scenario.grid.draw(canvas, line_distance))
        timesteps = 0
        while timesteps < time_limit:
            scenario.move()
            canvas.delete('all')
            time.sleep(1)#1 below 1000
            window.after(0, lambda: scenario.grid.draw(canvas, line_distance))
            window.update()
            timesteps += 1
            label.configure(text=str(timesteps)+' s')
            #print("---")
    
    btnStart = tk.Button(master=window, text="Start", bg='lightblue', width='18', command=lambda: start(window, canvas, scenario))
    btnStart.pack()
    
    #Below is an attempt to display explanations for the scenrio buttons. As this is rather cosmetic I suggest to do it at the end.
    #explanation = tk.Text(master=window, height=4, state=DISABLED)
    #explanation.insert(INSERT, "First scenario is one pedestrain walking on a straight line to the target.\n Second scenario is five pedestrians in a circle around the target.\n Third scenario is bottleneck.\n Fourth scenario is chicken test.\n")
    #explanation.pack(side=BOTTOM)
    
    window.mainloop()

In [10]:
if __name__ == "__main__":
    main()